In [ ]:
""""Thing to be done:
1 correlation
2 correlation with other pages
3 predictive model - neural network


1 compute another graph"""

# Near Real-Time Flu Estimation via Wikipedia

## Introduction



In the following notebook we are going to reproduce for Italy McIver paper et al. "Wikipedia usage estimates Prevalence of Influenza-like illness in theUnited States in near real time". In this paper we show a method of estimating, in near-real time, the level of influenza-like illness (ILI) in Italy by monitoring the rate of particular Wikipedia article views on a daily basis. We calculated on a weekly base the number of times certain influenza- or health-related Wikipedia articles were accessed and compared these data to one of the Italian health protection agency program called "Influnet".

The Notebook in the those three following sections:

* **Comparing Influnet and Wikipedia's Influenza click throught rate**
    * Retrivial and cleaning of the wikipedia pages:
        * Using 3rd party toolkit wikishark
        * Downloading the raw data from https://dumps.wikimedia.org/ (bonus point)
    * Plot the two scaled curves on the same figure
    * Compute Correlation
* **Comparing Influnet data with other Wikipedia's related pages:**
    * Retrivial and Cleaning
    * Plotting and correlation analysis among each other
* **Estimate Flu outbreaks:**
    * Lasso
    * Jesus
    
In the whole notebook we will make use of the following convetions:

## 1 - Comparison between Influnet and  Wikipedia Influenza 

In this section we are going to define the auxiliary functions that I defined in order to perform in order to perform data analysis in a more efficient way: 

In [2]:
import pandas as pd
import numpy as np
from os import listdir
import matplotlib.pyplot as plt
from scipy.stats import pearsonr 
import seaborn as sb

In [3]:
def importInflunet(path):
    '''
    Reads the Influnet data and creates a unique multiindex dataframe of the format
    
    (year,week) - incidence
    
    :param path: location of the influnet folder
    :return: compacted version of 
    '''
    
    df = pd.concat([pd.read_csv(path+t, names=["time", "incidence"], sep=" ", header=1, usecols=[0,4], decimal=",") for t in listdir(path)], ignore_index=True)
    df[["year","week"]] = df["time"].str.split("-", expand=True).astype(int)
    df.drop(["time"], axis=1, inplace=True)
    df = df.set_index(["year","week"])
    df.sortlevel(inplace=True)
    df = df.astype(float)
    df = df.loc[2008:]
    return df

def padInflunet(aux, year):
    '''
    The influnet dataset lacks information about the weeks that do not belog to the flu season (usally, but not necessarly, from week 17 to 40).
    This functions fills the dataset with empty position in order to match the wikipedia format.
    
    :param aux: Influnet dataframe from a specific year
    :param year: year of the previous Influnet dataframe
    :return: padded version of the original dataframe
    '''
    year_weeks = aux.index.values[-1]
    week_range = range(1,year_weeks+1)
    aux = aux.reindex(week_range, fill_value=0)
    aux["year"] = year
    aux["week"] = week_range
    
    aux.set_index(['year', 'week'], append=False, inplace=True)
    return aux


def getInflunet(path = "/home/aalto/Desktop/DE/hw2/influnet/data/"):
    '''
    import and reformat the original Influnet dataset
    
    :param path: 
    :return: clean and padded version of the Influnet dataset
    '''
    
    df = importInflunet();
    previous = None
    for x,y in df.index.values:
        if previous == None:
            df2 = reindexDF(df.loc[x], x)
        elif x != previous:
            df2 = df2.append(reindexDF(df.loc[x], x))
        previous = x
    return df

At this point we need the wikipedia pages views count in order to compare them with influnet. In order to satisy the requirement for the bonus point I wrote a simple script that downloads the files from https://dumps.wikimedia.org/, scan it to find the words we are intrested in and writes those entries of the files that we are intrested and write them on different files. 

Once those files are collected and stored on our disk we can call this function which loads the element passed them in memoery and group them by week.


In [4]:
def getWiki(wikiPages, path = "/home/aalto/PycharmProjects/digitalepidemiology/data/"):
    '''
    
    :param wikiPages: list of the wikipages that we want to analyze
    :param path: location of the downloaded wikipedia pages
    :return: 
    '''
    df = pd.DataFrame()
    for wikiPage in wikiPages:
        wiki2 = pd.read_csv(path+wikiPage+".csv", usecols=[0,1], parse_dates=[0], index_col=[0], header=None)
        wiki2 = wiki2.resample("W-Sun").sum()
        df[wikipage] = wiki2
    return df

Now to check the previous point we load the influnet dataset in memory and than show its plot

In [3]:
influnet = getInflunet()
influnet



x = wiki2.index[wiki2.index.year == 2010]
y = wiki2[wiki2.index.year == 2010]
    y2 = df2.loc[2010:][:len(x)]

comparePlots([influnet, wiki_influenza], [2010]) #needs to be in the right format

In [ ]:
wiki_influenza = getWiki("Influenza")

In [7]:
def comparePlots(elems, years):
    '''
    ta
    :param elems: list of dataframes that we  want to compare
    :param years: list of years that we want to anlayze
    :return: None (prompts the plot)
    '''
    for year in years:
        for elem in elems:
            plt.plot(x, y2/y2.max())
            plt.plot(x,y/y.max())

In [5]:
def getCorrelation(elems, years):
    '''
    
    :param elems: 
    :param years: 
    :return: 
    '''
    heatmap = pd.DataFrame([])
    for y in range(df2.index.min()[0], df2.index.max()[0]):
        try:
            heatmap = heatmap.append(list(pearsonr(df2.loc[year], wiki2[wiki2.index.year == y])))
        except ValueError:
            print y

    print heatmap
    sb.heatmap(pd.DataFrame(heatmap))
    plt.show()

SyntaxError: invalid syntax (<ipython-input-5-203aeb467653>, line 1)

# 2 - Other functions



At this point we are intrested to find if there is any other wikipedia pages that are able to gives us a better insghit about the topics